# NVIDEA Xavier NX GPS NMEA Data
<img src="https://developer.nvidia.com/sites/default/files/akamai/embedded/images/jetsonNX/Jetson_Xavier_NX-Developer_Kit-Front_Top_Right.png" alt="NVIDEA Xavier NX" style="width: 240px;"/>
I started this to understand how to pase the NMEAGPS data from serial port though the GPIO, *ttyTHS0*

I decided to use an Adafruit Ultimate GPS Breakout V3   
<img src="https://cdn-learn.adafruit.com/guides/cropped_images/000/000/077/medium640/thub.jpg?1520539843" alt="Adafruit Ultimate GPS" style="width: 240px;"/>

# Hardware and Dependancies
## Hardware
1. NVIDEA Xavier NX Developer Kit
2. Adafruit Ultimate GPS v3
3. Micro SD card (Min 32Gb recommended)

## Dependencies/software
1. Visual Studio Code (optional)
2. pySerial (Python)
3. virtualenv


# Wiring and testing
Wiring the GPS breakout board to the Xavier NX is very similar to the Raspberry Pi as it has the same GPIO so the same wiring can be use. 1 different point to note is that the GPIO is rotates 180 degreess in relation to the raspberry pi.

A useful resource to learn the GPIO pins can be found here at [Jetson Hacks(https://www.jetsonhacks.com/nvidia-jetson-xavier-nx-gpio-header-pinout/)

For the Raspberry Pi Wiring you can find it [here(https://learn.adafruit.com/assets/62851)

# Testing for data
Once everything is wired up if you check the serial port using the fllowing command in the terminal `sudo cat /dev/ttyTHS0` you will not get inny data from the GPS board and this is because the baud rate is not correct. The default baudrate is 11500, however, the Adafruit GPS operates on 9600. there is a simple fix to this.

In the terminal use the following command to set the baudrate  `stty -F /dev/ttyTHS0 9600` then to check that the baudrate has changed  simply type the command without the 9600 at the end and you shoulf get something like this
```
caprojects@XavierNX:~$ stty -F /dev/ttyTHS0 9600
caprojects@XavierNX:~$ stty -F /dev/ttyTHS0
speed 9600 baud; line = 0;
min = 0; time = 0;
-brkint -icrnl -imaxbel
-opost -onlcr
-isig -icanon -iexten -echo -echoe -echok -echoctl -echoke
caprojects@XavierNX:~$ 

```

Now type `sudo cat /dev/ttyTHS0` in the terminal again amd you will get the GPS data streaming. Now you know that the device and the Xavier is working witheach other and your wiring is correct.

This is not the way i wanted to collect and use the data so its time to moce on to python

# Python and Serial
The first thing I done was create a folder on the NVME drive to work from so all the data is in 1 place and be my work space. I then set up a Virthual Environment.

Open the folder that you want to work from, right click and select open in terminal. In the terminal window, enter `python3 -m venv env3` which will create your virual environment.

All my coding is done in [Visual Studio Code(https://code.visualstudio.com/) which is my go to code editor

While i understand that all what I am about to run though can be done with python packages, I decided to do this manually myself to understand GPS NMEA data more myself and how to decode it.

```python
import serial
from time import sleep

# Set port and set up serial
port = "/dev/ttyTHS0"
ser = serial.Serial(port, baudrate=9600)

while True:
    #delay for 1 sec, then wait for a message from serial and print the result
    sleep(1)
    ser.write(b'A')
    nbChars = ser.inWaiting()
    if nbChars>0:
        data = ser.read(nbChars)
        print(data)
```

This gets the data from the UART_1 *ttyTHS0* and the code does get the information ever 1 second.

I used this code to grabe some sample data to work on an NMEA decoder without the worry of ever chainging data.

a really handy resource for understanding NMEA sentences can be found here [GPS - NMEA sentence information(http://aprs.gids.nl/nmea/)

In [ ]:
# dataB is the raw byte string that is obtained from the GPS device
dataB = b'$GPGGA,233030.000,5629.4709,N,00255.3588,W,1,06,1.21,70.0,M,49.2,M,,*45\r\n$GPGSA,A,3,21,27,01,23,22,20,,,,,,,1.51,1.21,0.90*08\r\n$GPGSV,4,1,13,21,69,256,18,01,42,258,34,10,39,058,,27,35,138,22*7A\r\n$GPGSV,4,2,13,14,29,310,20,28,25,317,,22,24,204,33,32,22,105,19*76\r\n$GPGSV,4,3,13,23,10,044,18,20,07,047,17,30,02,281,,24,01,026,*7C\r\n$GPGSV,4,4,13,45,,,*7A\r\n$GPRMC,233030.000,A,5629.4709,N,00255.3588,W,0.15,144.55,040221,,,A*7B\r\n$GPZDA,233030.000,04,02,2021,,*50\r\n$GPGGA,233031.000,5629.4708,N,00255.3587,W,1,06,1.21,70.0,M,49.2,M,,*4A\r\n$GPGSA,A,3,21,27,01,23,22,20,,,,,,,1.51,1.21,0.90*08\r\n$GPRMC,233031.000,A,5629.4708,N,00255.3587,W,0.25,0.73,040221,,,A*72\r\n$GPZDA,233031.000,04,02,2021,,*51\r\n'

# Split the byte string into a list
dataD = dataB.split(b'$')

# List var for sorted data
gpsdata = [

# loop though the DataD list
for x in dataD:
    # ignore list entries that are blank
    if x != b'':        
        # decode byte string into UTF-8 and strip carrage returns
        t = str(x.decode('utf-8')).strip()
        
        # create list of remaning string
        t=t.split(',')
        
        # append it to  list to gpsdata list
        gpsdata.append(t)

# print the resutls
print(gpsdata)

What this code does is conver the following byte string into a more usable format

Original byte string

Decoded list formated data, Each line is a new list item

From here we can look up the NMEA sentence information and decode this further and format all the data in to a more usable format like json which i intend to do. By doing this, i can have all the information in a structure that I am happy and comfortable handling and can tweak it in the furture to take out information that I do not need if i so wish

# NMEA Sentences and Structure
From the decoded list data we can see that information for the following will be available
* GGA - Global Positioning System Fix Data
* GSA - GPS DOP and active satellites
* GSV - GPS Satellites in view
* RMC - Recommended minimum specific GPS/Transit data
* ZDA - Date & Time